# Read Data Sample

In [1]:
import pandas as pd
import numpy as np
import os
from collections import namedtuple
pd.set_option("display.max_rows",100)
#%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    kdd_test__2labels = pd.read_pickle("dataset/kdd_test__2labels.pkl")

    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels.pkl")
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels.pkl")
    

In [3]:
dataset.kdd_train_2labels.shape

(125973, 124)

In [4]:
dataset.kdd_test_2labels.shape

(22544, 124)

In [5]:
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['is_Normal','is_Attack']
    
    x_input = dataset.kdd_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.kdd_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.kdd_test_2labels.loc[:,output_columns_2labels]
    
    x_test__input = dataset.kdd_test__2labels.drop(output_columns_2labels, axis = 1)
    y_test_ = dataset.kdd_test__2labels.loc[:,output_columns_2labels]

    ss = pp.StandardScaler()

    x_train = ss.fit_transform(x_input)
    x_test = ss.transform(x_test_input)
    x_test_ = ss.transform(x_test__input)

    y_train = y_output.values
    y_test = y_test.values
    y_test_ = y_test_.values

preprocess.x_train.shape

(125973, 122)

In [6]:
import tensorflow as tf


In [7]:
class network(object):
    
    input_dim = 122
    classes = 2
    hidden_encoder_dim = 122
    hidden_layers = 1
    latent_dim = 18

    def __init__(self, classes, hidden_layers, num_of_features):
        self.classes = classes
        self.hidden_layers = hidden_layers
        self.latent_dim = num_of_features
            
    def build_layers(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        input_dim = self.input_dim
        classes = self.classes
        hidden_encoder_dim = self.hidden_encoder_dim
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        
        with tf.variable_scope("Input"):
            self.x = tf.placeholder("float", shape=[None, input_dim])
            self.y_ = tf.placeholder("float", shape=[None, classes])
            self.keep_prob = tf.placeholder("float")
            self.lr = tf.placeholder("float")
        
        with tf.variable_scope("Layer_Encoder"):

            hidden_encoder = tf.layers.dense(self.x, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            
            #hidden_encoder = tf.layers.dense(self.x, latent_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            #hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            
        with tf.variable_scope("Layer_Dense_Softmax"):
            self.y = tf.layers.dense(hidden_encoder, classes, activation=tf.nn.softmax)
            
        with tf.variable_scope("Loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.y_, logits = self.y))

            #loss = tf.clip_by_value(loss, -1e-1, 1e-1)
            #loss = tf.where(tf.is_nan(loss), 1e-1, loss)
            #loss = tf.where(tf.equal(loss, -1e-1), tf.random_normal(loss.shape), loss)
            #loss = tf.where(tf.equal(loss, 1e-1), tf.random_normal(loss.shape), loss)
            
            self.regularized_loss = loss
            correct_prediction = tf.equal(tf.argmax(self.y_, 1), tf.argmax(self.y, 1))
            self.tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "Accuracy")

        with tf.variable_scope("Optimizer"):
            learning_rate=self.lr
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        self.pred = tf.argmax(self.y, axis = 1)
        self.actual = tf.argmax(self.y_, axis = 1)

        # add Saver ops
        self.saver = tf.train.Saver()
        

In [8]:
import collections
import time

class Train:    
    
    result = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score', 'test_score_20', 'time_taken'])

    predictions = {}
    results = []
    
    best_acc = 0
    best_acc_global = 0
    
    def train(epochs, net, h,f, lrs):
        batch_iterations = 200
        train_loss = None
        Train.best_acc = 0
        os.makedirs("dataset/tf_dense_only_nsl_kdd/hidden layers_{}_features count_{}".format(epochs,h,f),
                    exist_ok = True)
        with tf.Session() as sess:
            #summary_writer_train = tf.summary.FileWriter('./logs/kdd/VAE/training', graph=sess.graph)
            #summary_writer_valid = tf.summary.FileWriter('./logs/kdd/VAE/validation')

            sess.run(tf.global_variables_initializer())
            start_time = time.perf_counter()
            for c, lr in enumerate(lrs):
                for epoch in range(1, (epochs+1)):
                    x_train, x_valid, y_train, y_valid, = ms.train_test_split(preprocess.x_train, 
                                                                              preprocess.y_train, 
                                                                              test_size=0.1)
                    batch_indices = np.array_split(np.arange(x_train.shape[0]), 
                                               batch_iterations)

                    for i in batch_indices:

                        def train_batch():
                            nonlocal train_loss
                            _, train_loss = sess.run([net.train_op, 
                                                               net.regularized_loss, 
                                                               ], #net.summary_op
                                                              feed_dict={net.x: x_train[i,:], 
                                                                         net.y_: y_train[i,:], 
                                                                         net.keep_prob:0.5, net.lr:lr})

                        train_batch()
                        #summary_writer_train.add_summary(summary_str, epoch)
                        while((train_loss > 1e4 or np.isnan(train_loss)) and epoch > 1):
                            print("Step {} | Training Loss: {:.6f}".format(epoch, train_loss))
                            net.saver.restore(sess, 
                                              tf.train.latest_checkpoint('dataset/tf_dense_only_nsl_kdd/hidden_layers_{}_features_count_{}'
                                                                         .format(epochs,h,f)))
                            train_batch()


                    valid_accuracy = sess.run(net.tf_accuracy, #net.summary_op 
                                                          feed_dict={net.x: x_valid, 
                                                                     net.y_: y_valid, 
                                                                     net.keep_prob:1, net.lr:lr})
                    #summary_writer_valid.add_summary(summary_str, epoch)


                    accuracy, pred_value, actual_value, y_pred = sess.run([net.tf_accuracy, 
                                                                   net.pred, 
                                                                   net.actual, net.y], 
                                                                  feed_dict={net.x: preprocess.x_test, 
                                                                             net.y_: preprocess.y_test, 
                                                                             net.keep_prob:1, net.lr:lr})
                    
                    accuracy_, pred_value_, actual_value_, y_pred_ = sess.run([net.tf_accuracy, 
                                                                   net.pred, 
                                                                   net.actual, net.y], 
                                                                  feed_dict={net.x: preprocess.x_test_, 
                                                                             net.y_: preprocess.y_test_, 
                                                                             net.keep_prob:1, net.lr:lr})

                    print("Step {} | Training Loss: {:.6f} | Validation Accuracy: {:.6f}".format(epoch, train_loss, valid_accuracy))
                    print("Accuracy on Test data: {}, {}".format(accuracy, accuracy_))

                    if accuracy > Train.best_acc_global:
                        Train.best_acc_global = accuracy
                        Train.pred_value = pred_value
                        Train.actual_value = actual_value
                        Train.pred_value_ = pred_value_
                        Train.actual_value_ = actual_value_
                        Train.best_parameters = "Hidden Layers:{}, Features Count:{}".format(h, f)

                    if accuracy > Train.best_acc:
                        Train.best_acc = accuracy

                        if not (np.isnan(train_loss)):
                            net.saver.save(sess, 
                                       "dataset/tf_dense_only_nsl_kdd/hidden_layers_{}_features_count_{}".format(h,f),
                                        global_step = epochs)
                        curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:, -1], "Prediction":pred_value})
                        Train.predictions.update({"{}_{}_{}".format((epoch+1)*(c+1),f,h):(curr_pred, 
                                                   Train.result((epoch+1)*(c+1), f, h, valid_accuracy, accuracy, accuracy_, time.perf_counter() - start_time))})

                        #Train.results.append(Train.result(epochs, f, h,valid_accuracy, accuracy))


In [9]:
import itertools

df_results = []
past_scores = []

class Hyperparameters:
#    features_arr = [2, 4, 8, 16, 32, 64, 128, 256]
#    hidden_layers_arr = [2, 4, 6, 10]

    def start_training():
        global df_results
        global past_scores
        Train.predictions = {}
        Train.results = []
    
        
        features_arr = [1, 8, 32, 122]
        hidden_layers_arr = [1, 3, 5]

        epochs = [10]
        lrs = [1e-5, 1e-5, 1e-6]
        for e, h, f in itertools.product(epochs, hidden_layers_arr, features_arr):
            print("Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
            n = network(2,h,f)
            n.build_layers()
            Train.train(e, n, h,f, lrs)
            
        dict1 = {}
        dict2 = []
        for k, (v1, v2) in Train.predictions.items():
            dict1.update({k: v1})
            dict2.append(v2)
        Train.predictions = dict1
        Train.results = dict2
        df_results = pd.DataFrame(Train.results)

        temp = df_results.set_index(['no_of_features', 'hidden_layers'])

        if not os.path.isfile('dataset/tf_dense_only_nsl_kdd_scores_all.pkl'):
            past_scores = temp
        else:
            past_scores = pd.read_pickle("dataset/tf_dense_only_nsl_kdd_scores_all.pkl")

        past_scores.append(temp).to_pickle("dataset/tf_dense_only_nsl_kdd_scores_all.pkl")

In [10]:
%%timeit -r 10
%%capture
Hyperparameters.start_training()

8min 5s ± 28.9 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [11]:
g = df_results.groupby(by=['no_of_features'])
idx = g['test_score'].transform(max) == df_results['test_score']
df_results[idx].sort_values(by = 'test_score', ascending = False)

,epoch,no_of_features,hidden_layers,train_score,test_score,test_score_20,time_taken
13,8,8,1,0.906414,0.880988,0.783291,7.756121
78,10,32,5,0.922289,0.869899,0.763207,26.090022
87,9,122,5,0.946896,0.858410,0.734599,23.537852
6,8,1,1,0.938403,0.791474,0.608186,8.059961


In [12]:
g = df_results.groupby(by=['no_of_features'])
idx = g['test_score_20'].transform(max) == df_results['test_score_20']
df_results[idx].sort_values(by = 'test_score_20', ascending = False)

,epoch,no_of_features,hidden_layers,train_score,test_score,test_score_20,time_taken
11,6,8,1,0.903239,0.880323,0.784051,5.512524
78,10,32,5,0.922289,0.869899,0.763207,26.090022
87,9,122,5,0.946896,0.858410,0.734599,23.537852
6,8,1,1,0.938403,0.791474,0.608186,8.059961


In [13]:
df_results.sort_values(by = 'test_score', ascending = False)

,epoch,no_of_features,hidden_layers,train_score,test_score,test_score_20,time_taken
13,8,8,1,0.906414,0.880988,0.783291,7.756121
12,7,8,1,0.905620,0.880944,0.783376,6.632713
11,6,8,1,0.903239,0.880323,0.784051,5.512524
78,10,32,5,0.922289,0.869899,0.763207,26.090022
74,6,32,5,0.920464,0.869721,0.763122,22.827152
72,4,32,5,0.909589,0.868879,0.761688,20.928807
79,11,32,5,0.902524,0.867770,0.759241,19.046752
87,9,122,5,0.946896,0.858410,0.734599,23.537852
10,5,8,1,0.825290,0.858011,0.761603,4.388568
86,8,122,5,0.945388,0.853220,0.725148,20.603261


In [14]:
pd.Panel(Train.predictions).to_pickle("dataset/tf_dense_only_nsl_kdd_predictions.pkl")
df_results.to_pickle("dataset/tf_dense_only_nsl_kdd_scores.pkl")


In [15]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=4)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j].round(4),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot(actual_value, pred_value):
    from sklearn.metrics import confusion_matrix
    cm_2labels = confusion_matrix(y_pred = pred_value, y_true = actual_value)
    plt.figure(figsize=[6,6])
    plot_confusion_matrix(cm_2labels, preprocess.output_columns_2labels, normalize = True,
                         title = Train.best_parameters)

In [16]:
plot(actual_value = Train.actual_value, pred_value = Train.pred_value)

Normalized confusion matrix
[[ 0.8616  0.1384]
 [ 0.0671  0.9329]]


In [17]:
plot(actual_value = Train.actual_value_, pred_value = Train.pred_value_)

Normalized confusion matrix
[[ 0.4596  0.5404]
 [ 0.0886  0.9114]]


In [18]:
psg = past_scores.groupby(by=['no_of_features', 'hidden_layers'])
psg.test_score.mean()

no_of_features  hidden_layers
1               1                0.760094
                3                0.713199
                5                0.625079
8               1                0.774272
                3                0.753633
                5                0.718815
32              1                0.775530
                3                0.769248
                5                0.789377
122             1                0.748377
                3                0.810419
                5                0.813828
Name: test_score, dtype: float64

In [19]:
past_scores

epoch  train_score  test_score  test_score_20  \
no_of_features hidden_layers                                                  
1              1                  2     0.605652    0.581308       0.310464   
               1                  3     0.628671    0.606991       0.351730   
               1                  4     0.956977    0.789567       0.605401   
               1                  5     0.855056    0.727156       0.497975   
               1                  6     0.894269    0.757807       0.551646   
               1                  7     0.914907    0.770538       0.574684   
               1                  8     0.933005    0.776260       0.585148   
               1                  9     0.943007    0.783446       0.596709   
               1                 10     0.951659    0.786861       0.601097   
               1                 11     0.954040    0.789035       0.604726   
8              1                  2     0.793301    0.771159       0.604810   
               1                  3     0.873948    0.816403       0.666920   
               1                  4     0.890618    0.825674       0.680084   
               1                  5     0.907684    0.833393       0.690127   
               1                  6     0.920702    0.833570       0.690127   
32             1                  2     0.709478    0.691049       0.449536   
               1                  3     0.793459    0.706574       0.467342   
               1                  4     0.957850    0.766324       0.558397   
               1                  5     0.852675    0.731946       0.496878   
               1                  6     0.961819    0.768985       0.561772   
               1                  8     0.965471    0.773510       0.569705   
               1                  9     0.940308    0.755900       0.539156   
               1                 10     0.944515    0.758960       0.544726   
               1                 11     0.950389    0.762509       0.551139   
122            1                  2     0.475155    0.627040       0.491224   
               1                  3     0.607001    0.724494       0.582532   
               1                  4     0.696618    0.753992       0.628945   
               1                  5     0.847357    0.805979       0.651055   
               1                  6     0.874663    0.817024       0.662700   
               1                  7     0.904112    0.819686       0.664388   
               1                  8     0.926179    0.827848       0.677722   
1              3                  2     0.534291    0.430758       0.181603   
               3                 11     0.921337    0.873181       0.768354   
               3                  4     0.926099    0.873669       0.769114   
               3                  6     0.931259    0.873891       0.768861   
8              3                  2     0.416733    0.407736       0.671224   
               3                  3     0.713447    0.707328       0.758481   
               3                  4     0.922686    0.884359       0.786329   
               3                  5     0.860851    0.856680       0.779578   
               3                  6     0.870773    0.860983       0.776709   
               3                  7     0.889903    0.867592       0.780000   
               3                  8     0.899270    0.874645       0.782278   
               3                  9     0.909351    0.880234       0.785654   
               3                 10     0.915383    0.881254       0.784979   
               3                 11     0.923480    0.883517       0.786835   
32             3                  2     0.786871    0.705066       0.625992   
               3                  3     0.873631    0.852555       0.736203   
               3                  4     0.893078    0.870032       0.766667   
               3                  7     0.921257    0.871185       0.764219   
 

In [20]:
psg = past_scores.groupby(by=['no_of_features', 'hidden_layers'])
psg.mean().test_score

no_of_features  hidden_layers
1               1                0.760094
                3                0.713199
                5                0.625079
8               1                0.774272
                3                0.753633
                5                0.718815
32              1                0.775530
                3                0.769248
                5                0.789377
122             1                0.748377
                3                0.810419
                5                0.813828
Name: test_score, dtype: float64